<a href="https://colab.research.google.com/github/jackcheng2020/Flowise/blob/main/Jack_cheng_aws_bedrock_rag_fednow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!python3 -m pip install -qU langchain elasticsearch boto3

# import modules
from getpass import getpass
from urllib.request import urlopen
from langchain.vectorstores import ElasticsearchStore
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.llms import Bedrock
from langchain.chains import RetrievalQA
import boto3
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 43.1 MB/s eta 0:00:00


In [30]:
default_region = "us-east-1"
AWS_ACCESS_KEY = getpass("AWS Acces key: ")
AWS_SECRET_KEY = getpass("AWS Secret key: ")

AWS_REGION = input(f"AWS Region [default: {default_region}]: ") or default_region

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY
)

AWS Acces key: ··········
AWS Secret key: ··········
AWS Region [default: us-east-1]: 


In [78]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="meta.llama2-70b-chat-v1", client=bedrock_client, model_kwargs={'max_gen_len':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

In [79]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://explore.fednow.org/resources/fraud-at-a-glance.pdf"

]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [80]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [81]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 85 documents loaded is 5702 characters.
After the split we have 570 documents more than the original 85.
Average length among 570 documents (after split) is 910 characters.


In [82]:
try:

    sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
    print("Sample embedding of a document chunk: ", sample_embedding)
    print("Size of the embedding: ", sample_embedding.shape)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution
    else:
        raise error

Sample embedding of a document chunk:  [-0.53125    -0.07861328 -0.33203125 ...  0.17382812  0.17675781
 -0.6640625 ]
Size of the embedding:  (1536,)


In [83]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

In [85]:
query = """How many transaction limits in Fednow?"""

In [86]:
query_embedding = bedrock_embeddings.embed_query(query)
np.array(query_embedding)

array([-0.68359375, -0.14941406,  0.27539062, ...,  0.171875  ,
        0.07519531, -0.2890625 ])

In [87]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: Network-level transaction limits  
The maximum amount per transaction a financial institution can send over the FedNow Service – amount set by the Federal Reserve.
Participant-level transaction limit 
Participants can set a lower transaction limit for credit transfers they initiate based on their organization’s risk policies.
Participant-defined negative lists  
Financial institutions may specify suspicious accounts their organizations can’t send to or receive from. 
PARTICIPANT REPORTING AND NOTIFICATION OF FRAUD
When FedNo
w participants confirm a transaction is fraudulent 
through their own investigation, they are required to report it to the FedNow Service. This information is used to strengthen the FedNow network and support counterparty fraud mitigation processes........
---
## Document 2: PROTECTING AGAINST INSTANT PAYMENT FRAUD
FedNow® Service risk management capabilities  
As with any type of payment,

In [88]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models.bedrock import ChatPromptAdapter
from langchain.schema import HumanMessage

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""


msg = HumanMessage(content=prompt_template)
prompt_template = ChatPromptAdapter().convert_messages_to_prompt('meta', [msg])

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [90]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
answer = qa({"query": query})
print(answer)

{'query': 'How many transaction limits in Fednow?', 'result': '  There are three transaction limits in FedNow:\n\n1. Network-level transaction limit: The maximum amount per transaction that a financial institution can send over the FedNow Service, set by the Federal Reserve.\n2. Participant-level transaction limit: The maximum amount per transaction that a participant can set for credit transfers they initiate, based on their organization’s risk policies.\n3. Participant-defined negative lists: Financial institutions may specify suspicious accounts that they cannot send to or receive from.', 'source_documents': [Document(page_content='Network-level transaction limits  \nThe maximum amount per transaction a financial institution can send over the FedNow Service – amount set by the Federal Reserve.\nParticipant-level transaction limit \nParticipants can set a lower transaction limit for credit transfers they initiate based on their organization’s risk policies.\nParticipant-defined negat